In [1]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 970.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import os
import pandas as pd
import json
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# загрузка библиотеки spacy и русского словаря
import spacy
nlp = spacy.load("ru_core_news_sm")

import ru_core_news_sm
nlp = ru_core_news_sm.load()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# указываем путь к рабочей папке
path = '/content/drive/MyDrive/hack_lct24'

# указываем имя файла с тестовым датасетом
test_filename = 'gt_test.csv'

In [5]:
# вспомогательная функция для определение номера токена в выборке по номеру его позиции
def find_pos(parser_array, n):
    pos = -1
    if n != -1:
        i = 0
        while i < len(parser_array):
            if parser_array[i]['pos'] == n:
                pos = i
                break
            i += 1
    return pos

# вспомогательная функция для корректировки отношения токена
def correct_relation(parser_array, n):
    i = find_pos(parser_array, n)
    if i != -1:
        if parser_array[i]['code'] == 'value':
            if parser_array[i]['relation'] != parser_array[i]['pos']:
                rel = correct_relation(parser_array, parser_array[i]['relation'])
                parser_array[i]['relation'] = rel
            else:
                rel = parser_array[i]['pos']
        else:
            rel = parser_array[i]['pos']
    else:
        rel = n

    return rel


# функция для поиска в заданном тексте интересующих слов
# ("скидка", числа, относящиеся к скидке, а также единицу измерения скидки: "процент" или "рубль")
def parser_text(inp_text):
    '''
    вход:
    inp_text - строка с входным текстом, в котором нуждной найти заданные слова

    выход:
    out_array - массив с маркерами, размерность которого равна кол-ву слов во входном тексте

    '''

    # обработаем входной текст с помощью spacy
    doc = nlp(inp_text)

    # запишем в inp_text_parser интересующие нас токены
    inp_text_parser = []
    for i in range(len(doc)):
        if doc[i].lemma_ in ['скидка', 'взнос', 'стоимость', 'цена', 'оплата', 'предоплата', 'ставка', 'ипотека', 'кредит']:
            # анализируемые сущности
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'entity'}]
        elif doc[i].lemma_ in ['процент', 'рубль', 'метр', 'квадрат', 'год', 'квартал', 'неделя', 'месяц', 'день', 'условие', 'этаж', 'корпус']:
            # измерения для чисел
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'measure', 'relation': -1}]
        elif doc[i].like_num:
            # числа
            inp_text_parser += [{'pos': i, 'text': doc[i].text, 'lemma': doc[i].lemma_, 'code': 'value', 'relation': doc[i].head.i}]

    # корректируем для чисел их отношение к измерению
    for i in range(len(inp_text_parser)):
        if inp_text_parser[i]['code'] == 'value':
            _ = correct_relation(inp_text_parser, inp_text_parser[i]['pos'])

    # удалим токены-числа и токены-измерения, если они относятся не к измерениям "процент" или "рубль"
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if inp_text_parser[i]['code'] == 'value':
            j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
            if (j == -1) or not(inp_text_parser[j]['lemma'] in ['процент', 'рубль']):
                _ = inp_text_parser.pop(i)
                length_parser -= 1
            else:
                i += 1
        elif (inp_text_parser[i]['code'] == 'measure') and not(inp_text_parser[i]['lemma'] in ['процент', 'рубль']):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1

    # расчитаем расстояния между сущностями и началом блоков чисел и измерения слева и справа
    # если между сущностями отсутствуют ичла и измерения, то установим -1
    length_parser = len(inp_text_parser)
    for i in range(length_parser):
        if inp_text_parser[i]['code'] == 'entity':
            left = -1
            if (i-1 >= 0) and (inp_text_parser[i-1]['code'] in ['value', 'measure']):
                left = abs(inp_text_parser[i]['pos'] - inp_text_parser[i-1]['pos'])
            inp_text_parser[i]['left'] = left
            right = -1
            if (i+1 < length_parser) and (inp_text_parser[i+1]['code'] in ['value', 'measure']):
                right = abs(inp_text_parser[i]['pos'] - inp_text_parser[i+1]['pos'])
            inp_text_parser[i]['right'] = right

    # удалим сущности для которых растояния и слева и справа -1
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if (inp_text_parser[i]['code'] == 'entity') and (inp_text_parser[i]['left'] == -1) and (inp_text_parser[i]['right'] == -1):
            _ = inp_text_parser.pop(i)
            length_parser -= 1
        else:
            i += 1

    # установим для чисел и измерений их отношение к сущностям
    length_parser = len(inp_text_parser)
    for i in range(length_parser):
        if inp_text_parser[i]['code'] == 'entity':
            if (inp_text_parser[i]['left'] == -1) or ((inp_text_parser[i]['left'] >= inp_text_parser[i]['right']) and (inp_text_parser[i]['right'] != -1)):
                j = i+1
                while (j < length_parser) and (inp_text_parser[j]['code'] != 'entity'):
                    j += 1
                if (i != j) and (j < length_parser) and (inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['right'] < inp_text_parser[j]['left']):
                    pos = inp_text_parser[i]['pos']
                    for k in range(i, j+1):
                        inp_text_parser[k]['relation'] = pos
                        inp_text_parser[k]['marker'] = 'I-value'
                    inp_text_parser[i+1]['marker'] = 'B-value'
            elif (inp_text_parser[i]['right'] == -1) or ((inp_text_parser[i]['left'] < inp_text_parser[i]['right']) and (inp_text_parser[i]['left'] != -1)):
                j = i-1
                while (j > 0) and (inp_text_parser[j]['code'] != 'entity'):
                    j -= 1
                if (i != j) and (j > 0) and (inp_text_parser[j]['code'] == 'entity') and (inp_text_parser[i]['left'] < inp_text_parser[j]['right']):
                    pos = inp_text_parser[i]['pos']
                    for k in range(j, i):
                        inp_text_parser[k]['relation'] = pos
                        inp_text_parser[k]['marker'] = 'I-value'
                    inp_text_parser[j+1]['marker'] = 'B-value'

    # удалим токены-числа и токены-измерения, если они относятся не к сущности "скидка"
    length_parser = len(inp_text_parser)
    i = 0
    while i < length_parser:
        if inp_text_parser[i]['code'] in ['value', 'measure']:
            j = find_pos(inp_text_parser, inp_text_parser[i]['relation'])
            if (j == -1) or not(inp_text_parser[j]['lemma'] in ['скидка']):
                _ = inp_text_parser.pop(i)
                length_parser -= 1
            else:
                i += 1
        elif inp_text_parser[i]['code'] == 'entity':
            if inp_text_parser[i]['lemma'] in ['скидка']:
                inp_text_parser[i]['marker'] = 'B-discount'
                i += 1
            else:
                _ = inp_text_parser.pop(i)
                length_parser -= 1
        else:
            i += 1

    # сформируем выходной массив
    out_array = ['O' for i in range(len(doc))]

    # расставим в выходном массиве маркеры для найденных слов
    for i in range(length_parser):
        pos = inp_text_parser[i]['pos']
        out_array[pos] = inp_text_parser[i]['marker']

    return out_array

In [6]:
# загружаем тестовый датасет по указанному пути
df_test = pd.read_csv(os.path.join(path, test_filename))

In [7]:
# выделим тестовые фразы в отдельный массив
test_texts = df_test['processed_text'].tolist()

In [8]:
# выполним parser_text для каждой тестовой фразы
# сохраняя результат обработки в выходной массив
test_arr = []
for test_txt in tqdm(test_texts):
    test_arr += [parser_text(test_txt)]

100%|██████████| 482/482 [01:01<00:00,  7.84it/s]


In [9]:
# обновим поле с метками в тестовом датасете
df_test['label'] = test_arr

In [11]:
# сохраним датасет в submission.csv по указанному пути
df_test.to_csv(os.path.join(path, 'submission.csv'), index=False)